# **Agricultural drought in central South America - trend analysis**

In [1]:
source("../methods_extRemes.r")

# method to obtain 95% confidence interval by bootstrapping
boot_ci <- function(mdl, rp, cov1, cov2, nsamp = 10000, lower = F) {
    
    n <- length(mdl$x)
    
    boot_sample <- rbind.fill(sapply(1:nsamp, function(i) {
        
        # resample & refit model
        boot_df <- mdl$cov.data[sample(1:n,n,replace = T),]
        suppressWarnings(boot_fit <- flnorm("precip", "gmst", boot_df, type = "fixeddisp", method = "MLE"))
        
        # compute required quantities for bootstrap (dispersion, PR, delta_I, delta_logI)
        pars_now <- sgev_pars(boot_fit, cov1)
        disp <- pars_now$scale / pars_now$loc
        
        rl <- return_level(boot_fit, rp, cov1, lower = lower)
        rlhist <- return_level(boot_fit, rp, cov2, lower = lower)
        
        pr <- prob_ratio(boot_fit, rl, cov1, cov2, lower = lower)
        
        dI <- delta_I(boot_fit, rp, cov1, cov2, rel = T, lower = lower)   
        dlogI <- (log(rl) - log(rlhist)) / log(rlhist) * 100
        
        as.data.frame(list("disp" = disp, "pr" = pr, "delta_I" = dI, "delta_logI" = dlogI))
        
    }, simplify = F))
    return(apply(boot_sample, 2, quantile, c(0.025, 0.975)))
}

---
## **CORDEX OND precip**

In [2]:
gsats = list('CCCma-CanESM2' = "CanESM2", 'CNRM-CERFACS-CNRM-CM5' = "CNRM-CM5", 'CSIRO-QCCCE-CSIRO-Mk3-6-0' = "CSIRO-Mk3-6-0", 'ICHEC-EC-EARTH' = "EC-EARTH",
             'IPSL-IPSL-CM5A-MR' = "IPSL-CM5A-LR", 'MIROC-MIROC5' = "MIROC5", 'MPI-M-MPI-ESM-LR' = "MPI-ESM-LR", "MOHC-HadGEM2-ES" = "HadGEM2-ES",
             "NCC-NorESM1-M" = "NorESM1-M", "NOAA-GFDL-GFDL-ESM2M" = "GFDL-ESM2G", "MPI-M-MPI-ESM-MR" = "MPI-ESM-MR")

In [ ]:
rp <- 20

cordex_pr_ond <- rbind.fill(sapply(list.files("ts", pattern = "pr-ond_SAM"), function(fnm) {
    
    set.seed(1)
    
    # print(fnm)
    if(grepl("r12i1p1", fnm)) {
        gmst_fnm <- '/home/clair/wwa/00_gsat/cordex_tas_Amon_EC-EARTH_rcp85_r12i1p1.dat'
    } else {
        gcm <- strsplit(fnm, "_")[[1]][4]
        gmst_fnm <- list.files("/home/clair/wwa/00_gsat", pattern = gsats[[gcm]], full.names = T)
    }
        
    if(length(gmst_fnm) == 0) {return(NULL)}
    
    # load data + GCM GMST into a single dataframe
    df <- read.csv(paste0("ts/", fnm), comment.char = "#", header = F, sep = " ", col.names = c("year", "precip"))
    gmst <- read.fwf(gmst_fnm, comment.char = "#", widths = c(5,15), col.names = c("year", "gmst"))
    df <- merge(gmst, df)
    row.names(df) <- df$year
    
    event_gmst <- df["2022", "gmst"]
    
    mdl_eval <- flnorm("precip", "gmst", df[df$year >= 1980 & df$year <= 2022,], type = "fixeddisp", method = "MLE")  # obs period only
    mdl_attr <- flnorm("precip", "gmst", df[df$year <= 2022,], type = "fixeddisp", method = "MLE")                    # up to present day
    mdl_proj <- flnorm("precip", "gmst", df[df$year <= 2050,], type = "fixeddisp", method = "MLE")                    # up to 2050
    
    # parameters from evaluation fit: just dispersion (& event value for reference)
    eval_pars_now <- sgev_pars(mdl_eval, event_gmst)
    eval_disp <- eval_pars_now$scale / eval_pars_now$loc                                # needs bootstrapping
    eval_rl <- return_level(mdl_eval, rp, event_gmst, lower = T)
    
    # parameters from attribution fit:
    attr_rl <- return_level(mdl_attr, rp, event_gmst, lower = T)
    attr_rlhist <- return_level(mdl_attr, rp, event_gmst - 1.2, lower = T)
    attr_rphist <- return_period(mdl_attr, attr_rl, event_gmst - 1.2, lower = T)
    attr_pars_now <- sgev_pars(mdl_attr, event_gmst)
    attr_disp <- attr_pars_now$scale / attr_pars_now$loc       
    attr_pr <- prob_ratio(mdl_attr, attr_rl, event_gmst, event_gmst - 1.2, lower = T)
    attr_dI <- delta_I(mdl_attr, rp, event_gmst, event_gmst - 1.2, rel = T, lower = T)   
    attr_dlogI <- (log(attr_rl) - log(attr_rlhist)) / log(attr_rlhist) * 100
    
    # parameters from future projection:
    proj_rl <- return_level(mdl_proj, rp, event_gmst, lower = T)
    proj_rlfut <- return_level(mdl_proj, rp, event_gmst + 0.8, lower = T)
    proj_rpfut <- return_period(mdl_proj, proj_rl, event_gmst + 0.8, lower = T)
    proj_pars_now <- sgev_pars(mdl_proj, event_gmst)
    proj_disp <- proj_pars_now$scale / proj_pars_now$loc       
    proj_pr <- prob_ratio(mdl_proj, proj_rl, event_gmst + 0.8, event_gmst, lower = T)
    proj_dI <- delta_I(mdl_proj, rp, event_gmst + 0.8, event_gmst, rel = T, lower = T)   
    proj_dlogI <- (log(proj_rlfut) - log(proj_rl)) / log(proj_rl) * 100
    
    # bootstrap (has to be fitted for each model individually to ensure sampling of correct years)...
    ci_eval <- boot_ci(mdl_eval, rp = rp, cov1 = event_gmst, cov2 = event_gmst - 1.2, nsamp = 10000, lower = T)
    ci_attr <- boot_ci(mdl_attr, rp = rp, cov1 = event_gmst, cov2 = event_gmst - 1.2, nsamp = 10000, lower = T)
    ci_proj <- boot_ci(mdl_proj, rp = rp, cov1 = event_gmst + 0.8, cov2 = event_gmst, nsamp = 10000, lower = T)
    
    # return results
    cbind(model = gsub(".+-ond_", "", gsub(".dat", "", fnm)),
          as.data.frame(list("eval_disp" = eval_disp, "eval_disp_lower" = ci_eval["2.5%","disp"], "eval_disp_upper" = ci_eval["97.5%","disp"],
                             "attr_disp" = attr_disp, "attr_disp_lower" = ci_attr["2.5%","disp"], "attr_disp_upper" = ci_attr["97.5%","disp"],
                             "attr_value" = attr_rl, "attr_logvalue" = log10(attr_rl), "attr_rp" = rp, "attr_rp_hist" = attr_rphist, 
                             "attr_pr" = attr_pr, "attr_pr_lower" = ci_attr["2.5%","pr"], "attr_pr_upper" = ci_attr["97.5%","pr"],
                             "attr_dI" = attr_dI, "attr_dI_lower" = ci_attr["2.5%","delta_I"], "attr_dI_upper" = ci_attr["97.5%","delta_I"],
                             "attr_dlogI" = attr_dlogI, "attr_dlogI_lower" = ci_attr["2.5%","delta_logI"], "attr_dlogI_upper" = ci_attr["97.5%","delta_logI"],
                             "proj_pr" = proj_pr, "proj_pr_lower" = ci_proj["2.5%","pr"], "proj_pr_upper" = ci_proj["97.5%","pr"],
                             "proj_dI" = proj_dI, "proj_dI_lower" = ci_proj["2.5%","delta_I"], "proj_dI_upper" = ci_proj["97.5%","delta_I"],
                             "proj_dlogI" = proj_dlogI, "proj_dlogI_lower" = ci_proj["2.5%","delta_logI"], "proj_dlogI_upper" = ci_proj["97.5%","delta_logI"]
                             )))
    
}, simplify = F))
write.csv(cordex_pr_ond, "cordex-pr-ond-results.csv", row.names = F)

---
## **FLOR & AM2 OND precip**